## Overview

The Files API is part of the Gemini API family. This new API empowers developers to upload files for Gemini to use in multi-model scenarios. The files API is the most stable and least error prone method of unlocking multi-model support in your applications using Gemini.


> The Files API is currently in early access. Features and endpoints may change. Please use with caution in production environments.

## Setup


### Authentication Overview

Files API uses API keys for authentication and access. Files uploaded via the API belong to the cloud project used during the request. As such, take extra caution in keeping your API keys secure. Publicly exposing the credentials can result in your data being compromised. For best practices on securing API keys, refer to Google's [documentation](https://support.google.com/googleapi/answer/6310037).

### Setup your API key

Before you can use the Gemini API, you must first obtain an API key. If you don't already have one, create a key with one click in Google AI Studio.

<a class="button button-primary" href="https://makersuite.google.com/app/apikey" target="_blank" rel="noopener noreferrer">Get an API key</a>

In Colab, add the key to the secrets manager under the "🔑" in the left panel. Give it the name `GOOGLE_API_KEY`.

Once you have the API key, pass it to the SDK. You can do this in two ways:

* Put the key in the `GOOGLE_API_KEY` environment variable (the SDK will automatically pick it up from there).
* Pass the key to `genai.configure(api_key=...)`



In [ ]:
# Used to securely store your API key
from google.colab import userdata

# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

### Build Google API Client for Generative Language API

NOTE: Generative Language SDK support for Files API is not yet available. For now, we will use the public [google-api-python-client](https://github.com/googleapis/google-api-python-client) to build a client library from the discovery docs.

In [ ]:
# Fetch the discovery docs for the Generative Language API service.
from googleapiclient.discovery import build
import googleapiclient
import requests

DISCOVERY_URL = f'https://generativelanguage.googleapis.com/$discovery/rest?version=v1beta&key={GOOGLE_API_KEY}';
discovery_docs = requests.get(DISCOVERY_URL).content
genai_service = googleapiclient.discovery.build_from_document(discovery_docs, developerKey=GOOGLE_API_KEY)

## Upload files

Once we have the client service built, we are now ready to create our file.


The Files API lets you upload a variety of multi-modal MIME types including images. The API is intended only for model inference calls, as such the API has the following characteristics:

* Can only be referenced for model inference calls
* Automatic deletion after 2 days
* Maximum 2GB per file, 20GB limit per project
* No downloads allowed


To get started, upload the file you'd like to use for an LLM model inference call by using the file icon on the left sidebar, then the upload icon, as shown in the screenshot below.

<img width=400 src="https://ai.google.dev/tutorials/images/colab_upload.png">

In [ ]:
import mimetypes

class File:
  def __init__(self, file_path: str, display_name: str = None,
               mimetype: str = None, uri = None):
    self.file_path = file_path
    if display_name:
      self.display_name = display_name
    # Detect mimetype if not specified
    self.mimetype = mimetype if mimetype else mimetypes.guess_type(file_path)[0]
    self.uri = uri

  def set_file_uri(self, uri):
    self.uri = uri

# Reference the file uploaded
uploaded_file_name = "gemini.png" # @param {type:"string"}
file_display_name = "Gemini Logo" # @param {type:"string"}
# Create a file object
files_to_upload = [
    File(file_path=uploaded_file_name,
         display_name=file_display_name)]

# Upload the files to the API
uploaded_files = []
print(f'Uploading {len(files_to_upload)} files...')
for file in files_to_upload:
  response = genai_service.media().upload(
      media_body=file.file_path,
      media_mime_type = file.mimetype,
      body={"file": {"display_name": file.display_name}}).execute()
  file.set_file_uri(response["file"]["uri"])
  uploaded_files.append(file)


print(f"Completed file uploads!\n\nUploaded: {len(uploaded_files)} files")

## List Files

After uploading the file, you can verify it was successfully uploaded by making a GetFile or ListFiles request.

In [ ]:
# List files uploaded in the API
response = genai_service.files().list().execute()
print(response)

## Generate Content

After the files have been uploaded, you can now make GenerateContent requests with the file URIs provided in the response.

For video inference calls, provide each image frame in the format of 2 parts: the timestamp and image URI like bwlow:

```
part { text = "00:00" }
part { fileData = fileData {
  fileUri = "https://generativelanguage.googleapis.com/v1/files/frame-0"
  mimeType = "image/jpeg"
}}


In [ ]:
# Make GenerateContent Request
def makeGenerateContentRequest(prompt, files):
  generateContent = {"contents": [{ "parts": [{ "text": prompt }]}]}
  for file in files:
    # for images, use the following:
    generateContent["contents"][0]["parts"].append(makeImagePart(file))
    # for video frames, use the following:
    # generateContent["contents"][0]["parts"].extend(makeVideoPart(file))
  return generateContent

def makeImagePart(file):
  return [
      {"file_data": {"file_uri": file.uri, "mime_type": file.mimetype }}]

def makeVideoPart(file):
  return [
      {"text": file.timestamp},
      {"file_data": {"file_uri": file.uri, "mime_type": file.mimetype }}]

prompt = "Describe the image with a creative description" # @param {type:"string"}
model = "models/gemini-1.5-pro-latest" # @param ["models/gemini-1.5-pro-latest", "models/gemini-1.0-pro-vision-latest"]
response = genai_service.models().generateContent(
    model = model,
    body = makeGenerateContentRequest(prompt, uploaded_files)).execute()
print(response)

## Delete Files

If you no longer need the file, you can either allow the file to be automatically deleted after 2 days or manually delete it using `DeleteFile()`.

In [ ]:
# Delete the files with its resource name
print(f'Deleting {len(files_to_upload)} files...')
for file in uploaded_files:
  resource = file.uri.split("/files/")[-1]
  response = genai_service.files().delete(name=f"files/{resource}").execute()
  print(f'Deleted {file.file_path} as URI {file.uri}')

print(f"Completed deleting files!\n\nDeleted: {len(uploaded_files)} files")